
# Profitable App Profiles for the App Store and Google Play Markets

Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

## Opening and Exploring the Data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

* A data set containing data about approximately 10,000 Android apps from Google Play
* A data set containing data about approximately 7,000 iOS apps from the App Store

Let's start by opening the two data sets and then continue with exploring the data.



In [1]:
#Read datasets

##Import the function from the reader package
from csv import reader
import pandas as pd

##Open the files
file_andr = "googleplaystore.csv" #Android
android = pd.read_csv(file_andr)

file_ios = "AppleStore.csv"
iphone = pd.read_csv(file_ios)

print(iphone.head(2))
print("/n")
print("Number of rows of iphone",len(iphone))
print("/n")
print(android.head(2))
print("/n")
print("Number of rows of android:", len(android))

          id track_name  size_bytes currency  price  rating_count_tot  \
0  284882215   Facebook   389879808      USD    0.0           2974676   
1  389801252  Instagram   113954816      USD    0.0           2161558   

   rating_count_ver  user_rating  user_rating_ver    ver cont_rating  \
0               212          3.5              3.5   95.0          4+   
1              1289          4.5              4.0  10.23         12+   

         prime_genre  sup_devices.num  ipadSc_urls.num  lang.num  vpp_lic  
0  Social Networking               37                1        29        1  
1      Photo & Video               37                0        29        1  
/n
Number of rows of iphone 7197
/n
                                              App        Category  Rating  \
0  Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                             Coloring book moana  ART_AND_DESIGN     3.9   

  Reviews Size  Installs  Type Price Content Rating  \
0     159  1

The Apple Store datasat contain 7197 rows and 16 columns 
while Google Play has 10841 rows but three columns less. 
As it can be seen the variables in the two datasets are
not the same. Thus, *name of the app*, *category*, *rating*, 
*genre*, *installs* and *number of users for current and all versions*
are the first main suspects as the variable to move forward with 
in the analysis. For more detailed information about the datasets
and the variables here is the link to the  https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home

## Data cleaning

The first and main criterias for the apps that we should analyze are that they should be ** free ** and ** english** in order to match the request from the deveolpment team in order to analyse * apples-to-apples *.

### Descriptive statistics

#### Google PlayStore initial observations
Two main observations can be drawn from the table above. The ratings column has ** missing values ** as its top frequency value and 60% of the app have a ** rating ** of 0. Also, there are 9660 unique apps which suggests that around * 1100 * could be duplicates. 

To check whether the app is in english or not, only the variable with the ** app name ** can be helpful. As for * free apps *, 97% of the dataset has free apps, so good news! 

##### Irregular values
From the ratings columns, there is **a maximum value of 19** which is clearly an error.

In [2]:
#Look at the statistics summary
android.describe()

,Rating
count,9367.000000
mean,4.193338
std,0.537431
min,1.000000
25%,4.000000
50%,4.300000
75%,4.500000
max,19.000000


In [3]:
#Remove the outlier row which 19 has app rating
print("Before",len(android))
android = android.drop(10472)

print("After removing the gross outlier", len(android))

Before 10841
After removing the gross outlier 10840


#### Duplicates

Clearly there are some duplicates (1181 to be exact) for app names. The criterion to decide which row to keep will be ** number of reviews **, because the biggeer the reviews the more recent the row of data.

In [4]:
# Duplicate rows
## Assign to the new column the duplicate Boolean
android['is_duplicate'] = android['App'].duplicated() 

print('Number of duplicates:', len(android[android['is_duplicate'] == True]))
print('Examples:', android[android['is_duplicate'] == True].sort_values('App').head())

Number of duplicates: 1181
Examples:                                App            Category  Rating Reviews  Size  \
1407         10 Best Foods for You  HEALTH_AND_FITNESS     4.0    2490  3.8M   
2543    1800 Contacts - Lens Store             MEDICAL     4.7   23160   26M   
2385    2017 EMRA Antibiotic Guide             MEDICAL     4.4      12  3.8M   
1434  21-Day Meditation Experience  HEALTH_AND_FITNESS     4.4   11506   15M   
5415       365Scores - Live Scores              SPORTS     4.6  666246   25M   

         Installs  Type   Price Content Rating            Genres  \
1407     500,000+  Free       0   Everyone 10+  Health & Fitness   
2543   1,000,000+  Free       0       Everyone           Medical   
2385       1,000+  Paid  $16.99       Everyone           Medical   
1434     100,000+  Free       0       Everyone  Health & Fitness   
5415  10,000,000+  Free       0       Everyone            Sports   

           Last Updated Current Ver   Android Ver  is_duplicate  
1407  F

In [5]:
#Example of duplicates
android[android['App'] == "Instagram"]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,is_duplicate
2545,Instagram,SOCIAL,4.5,66577313,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device,False
2604,Instagram,SOCIAL,4.5,66577446,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device,True
2611,Instagram,SOCIAL,4.5,66577313,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device,True
3909,Instagram,SOCIAL,4.5,66509917,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device,True


In [6]:
### The Google Play data set ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### The App Store data set ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

del android[10472]

In [7]:
#Create a dictionary of app with most recent reviews
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
len(reviews_max) #as expected

9659



Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

* We start by initializing two empty lists, android_clean and already_added.
* We loop through the android data set, and for every iteration:
- We isolate the name of the app and the number of reviews.
- We add the current row (app) to the android_clean list, and the app name (name) to the already_cleaned list if:

   * The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
   * The name of the app is not already in the already_added list. 

*We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.*



In [8]:
#Remove the duplicates
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

#### Apple Store Dataset initial observations
For the Apple store dataset there are no duplicates. Another good news is that ** 32% ** of the Apple Store dataset has ratings of 0. Differently from the Android dataset the language column can easily filter out the non-english apps (57 different languages avaialble). As for free apps, they make up only 56% of the dataset.

In [9]:
#describe shows only numerical variables
iphone.describe()

,id,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
count,7.197000e+03,7.197000e+03,7197.000000,7.197000e+03,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000
mean,8.631310e+08,1.991345e+08,1.726218,1.289291e+04,460.373906,3.526956,3.253578,37.361817,3.707100,5.434903,0.993053
std,2.712368e+08,3.592069e+08,5.833006,7.573941e+04,3920.455183,1.517948,1.809363,3.737715,1.986005,7.919593,0.083066
min,2.816565e+08,5.898240e+05,0.000000,0.000000e+00,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000
25%,6.000937e+08,4.692275e+07,0.000000,2.800000e+01,1.000000,3.500000,2.500000,37.000000,3.000000,1.000000,1.000000
50%,9.781482e+08,9.715302e+07,0.000000,3.000000e+02,23.000000,4.000000,4.000000,37.000000,5.000000,1.000000,1.000000
75%,1.082310e+09,1.819249e+08,1.990000,2.793000e+03,140.000000,4.500000,4.500000,38.000000,5.000000,8.000000,1.000000
max,1.188376e+09,4.025970e+09,299.990000,2.974676e+06,177050.000000,5.000000,5.000000,47.000000,5.000000,75.000000,1.000000


### Removing non-English apps

Each charachter in a string is associated with a number. The numbers corresponding to the **characters we commonly use in an English text are all in the range 0 to 127**, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters, otherwise it doesn't.

In [10]:
#Remove non-english apps

#Create a function to detect if the charachter of any app name are non-english
def is_english(input):
    count = 0
    for charachter in input:
        if (ord(charachter) > 127):
            count += 1
#more than three, because emoji or special symbol can mess the filter    
#minimizing errors of missclassifying english apps as non apps
    if count > 3:                 
        return False
    else:    
        return True

#Looping over each app name for both datasets
android_fresh = [] #list to append updated apps
ios_fresh = []

for app in android_clean:
    name = str(app[0])
    if (is_english(name) == True):
        android_fresh.append(app)
        
for app in ios:
    name = app[1]
    if (is_english(name) == True):
        ios_fresh.append(app)
        
        
print('Google PlayStore English app number of rows:',len(android_fresh))
print('Apple Store English app number of rows:', len(ios_fresh))

Google PlayStore English app number of rows: 9614
Apple Store English app number of rows: 6183


### Isolating free apps

As mentioned above, this will be the last step of *data cleaning* for now, because we sattisfy the initial requirements from the development team.

In [11]:
free_android = []
free_ios = []

for app in android_fresh:
    price = app[7]
    if price == '0':
        free_android.append(app)
        
        
for app in ios_fresh:
    price = app[4]
    if price == '0.0':
        free_ios.append(app)
    
print('Free apps number of observations - Google Play Store:', len(free_android))
print('Free apps number of observations - Apple Play Store:', len(free_ios))

Free apps number of observations - Google Play Store: 8864
Free apps number of observations - Apple Play Store: 3222


### Data Cleaning Wrap-up

  * Removed inaccurate data
  * Removed duplicate app entries
  * Removed non-English apps
  * Isolated the free apps

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

  1. Build a minimal Android version of the app, and add it to Google Play.
  2. If the app has a good response from users, we then develop it further.
  3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our ** end goal is to add the app on both the App Store and Google Play **, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.


In [12]:
#function that creates frequency of ganres
def freq_table(data, index):
    freq_table = {}
    total = 0
    for row in data:
        total += 1
        value = row[index]
        if value in freq_table:
            freq_table[value] += 1
        else:
            freq_table[value] = 1
    
    #express them as %
    table_percentage = {}
    for key in freq_table:
        percentage = ((freq_table[key]/ total)*100)  
        table_percentage[key] = percentage
    return table_percentage
        
    
    


#arrange frequency table into tuple and sortes it out
def display_table(data, index):
    table = freq_table(data, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

#### Key notes from frequency table (Ganre & Category)

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

In [13]:
#Display the frequnecy table for google play store apps
display_table(free_android, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

#### Diving deeper (which categories have the most users?)

We will check this out by dividing the frequency table with the number of installs found in the *Installs* column for the Google store dataset. While for the Apple Store dataset we will use the *rating_count_tot* as a workaround.


In [14]:
#Isolate user rating per app genre (Apple)
ios_genre = freq_table(free_ios, -5)

#loop over genre in ios_genre
for genre in ios_genre:
    total = 0
    len_genre = 0 
    #loop over apple dataset rows
    for app in free_ios:
        genre_app = app[-5]
        if genre == genre_app:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    #outside the nested loop
    avg_rating = total/len_genre #number of ratings by number of same category
    
    print("Genre is:", genre)
    print("/n")
    print("Average number of ratings is:", avg_rating)
    




Genre is: Health & Fitness
/n
Average number of ratings is: 23298.015384615384
Genre is: Weather
/n
Average number of ratings is: 52279.892857142855
Genre is: Education
/n
Average number of ratings is: 7003.983050847458
Genre is: Book
/n
Average number of ratings is: 39758.5
Genre is: Navigation
/n
Average number of ratings is: 86090.33333333333
Genre is: Travel
/n
Average number of ratings is: 28243.8
Genre is: Photo & Video
/n
Average number of ratings is: 28441.54375
Genre is: Utilities
/n
Average number of ratings is: 18684.456790123455
Genre is: Lifestyle
/n
Average number of ratings is: 16485.764705882353
Genre is: Food & Drink
/n
Average number of ratings is: 33333.92307692308
Genre is: Entertainment
/n
Average number of ratings is: 14029.830708661417
Genre is: Shopping
/n
Average number of ratings is: 26919.690476190477
Genre is: Social Networking
/n
Average number of ratings is: 71548.34905660378
Genre is: Business
/n
Average number of ratings is: 7491.117647058823
Genre is: N

As it can be seen above, the Genres for the Apple Data set that have the ** most users** are *entertainment and informative apps* like: 
 * Navigation - 86,090
 * Reference - 74,942
 * Social Networking - 71, 548
 * Music - 57, 326
 * Weather - 52, 221
 * Book - 39, 271
 * Food & Drinks - 33, 300
 * Finance - 31, 200
 
 Other app which big user rate base include *Games, Health & fitness, Catalogs, Photo and video.*
 

In [25]:
#Analyse the biggest user based genre - Navigation
for app in free_ios:
    genre = app[-5]
    n_ratings = app[5]
    name = app[1]
    if genre == "Navigation":
        print(name, n_ratings)

Waze - GPS Navigation, Maps & Real-time Traffic 345046
Google Maps - Navigation & Transit 154911
Geocaching® 12811
CoPilot GPS – Car Navigation & Offline Maps 3582
ImmobilienScout24: Real Estate Search in Germany 187
Railway Route Search 5


As we doubted, *the average rating* for the ganre ** Navigation ** is heavily uplifted from *Waze* and *Google Maps* which contribute close to half a milion users.

Similar to the Navigation genre, **Social Networking** genre is very imbalanced with two extreme edges. On the upper part, apps like *Facebook, Skype, Whatsapp, Pinterest, Messenger* have like 5 million users which skew the ratings up while the rest of the apps combine together for like 400,000 users.

In [37]:
for app in free_ios:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

* Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

* Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

* Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

In [16]:
display_table(free_android, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


In [44]:
#Repaeat the procedure for Google store dataset
android_genre = freq_table(free_android, 1) #frequency per ganre table

for category in android_genre:
    total = 0
    len_category = 0
    for app in free_android:
        category_app = app[1]
        installs = app[5]
        if category == category_app:
            #replace + and , in order to convert installs into a float
            installs = installs.replace('+', '')
            installs = installs.replace(',', '')
            total += float(installs)
            len_category += 1
            
    #compute the average
    avg_n_rating = total / len_category
    print("Average number of installs for", category, "is:", avg_n_rating)
            

Average number of installs for PRODUCTIVITY is: 16787331.344927534
Average number of installs for ENTERTAINMENT is: 11640705.88235294
Average number of installs for SHOPPING is: 7036877.311557789
Average number of installs for BUSINESS is: 1712290.1474201474
Average number of installs for TRAVEL_AND_LOCAL is: 13984077.710144928
Average number of installs for DATING is: 854028.8303030303
Average number of installs for HOUSE_AND_HOME is: 1331540.5616438356
Average number of installs for HEALTH_AND_FITNESS is: 4188821.9853479853
Average number of installs for COMMUNICATION is: 38456119.167247385
Average number of installs for AUTO_AND_VEHICLES is: 647317.8170731707
Average number of installs for BOOKS_AND_REFERENCE is: 8767811.894736841
Average number of installs for BEAUTY is: 513151.88679245283
Average number of installs for EVENTS is: 253542.22222222222
Average number of installs for PHOTOGRAPHY is: 17840110.40229885
Average number of installs for LIFESTYLE is: 1437816.2687861272
Avera

In [55]:
for app in free_android:
    installs = app[5]
    installs = installs.replace('+', '')
    installs = installs.replace(',', '')
    if app[1] == "BOOKS_AND_REFERENCE":
        if float(installs) < 100000000:
            print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
English translation from Bengali : 1

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.
